In [1]:
import tensorflow as tf
from tensorflow.feature_column import embedding_column, numeric_column, categorical_column_with_identity
import pandas as pd
import matplotlib.pyplot as plt
print(tf.__version__)

1.13.1


In [438]:
TRAIN_PATH = "/Users/bryanwu/Desktop/taxi_fare_prediction/new-york-city-taxi-fare-prediction/train.csv"
df = pd.read_csv(TRAIN_PATH, nrows=100_000, parse_dates=['pickup_datetime'])

In [452]:
def preprocess(df, mean=None, std=None):
    #remove negative fare_amount
    df = df[df['fare_amount'] > 0]
    
    #remove missing values
    df.dropna(how='any', axis='rows')
    
    #remove location outliers, ensuring locations are within the range seen in the train set
    def select_within_boundingbox(df, BB=(-74.5, -72.8, 40.5, 41.8)):
        return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
               (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
               (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
               (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])
    
    df = df[select_within_boundingbox(df)]
    
    #remove trips in the water
    def remove_datapoints_from_water(df):
        def lonlat_to_xy(longitude, latitude, dx, dy, BB):
            return (dx*(longitude - BB[0])/(BB[1]-BB[0])).astype('int'), \
                   (dy - dy*(latitude - BB[2])/(BB[3]-BB[2])).astype('int')

        # define bounding box
        BB = (-74.5, -72.8, 40.5, 41.8)

        # read nyc mask and turn into boolean map with
        # land = True, water = False
        nyc_mask = plt.imread('https://aiblog.nl/download/nyc_mask-74.5_-72.8_40.5_41.8.png')[:,:,0] > 0.9

        # calculate for each lon,lat coordinate the xy coordinate in the mask map
        pickup_x, pickup_y = lonlat_to_xy(df.pickup_longitude, df.pickup_latitude, 
                                          nyc_mask.shape[1], nyc_mask.shape[0], BB)
        dropoff_x, dropoff_y = lonlat_to_xy(df.dropoff_longitude, df.dropoff_latitude, 
                                          nyc_mask.shape[1], nyc_mask.shape[0], BB)    
        # calculate boolean index
        idx = nyc_mask[pickup_y, pickup_x] & nyc_mask[dropoff_y, dropoff_x]

        # return only datapoints on land
        return df[idx]
    
    df = remove_datapoints_from_water(df)
    
    #Step 2: preprocess features
    weekday = df.pickup_datetime.apply(lambda t: t.weekday) 
    month = df.pickup_datetime.apply(lambda t: t.month) - 1
    hour = df.pickup_datetime.apply(lambda t: t.hour)
    minute = df.pickup_datetime.apply(lambda t: t.minute)
    dayofyear= df.pickup_datetime.apply(lambda t: t.dayofyear)
    nyc_mask = plt.imread('https://aiblog.nl/download/nyc_mask-74.5_-72.8_40.5_41.8.png')[:,:,0] > 0.9
    #translate longitude/latitude coordinate into image xy coordinate
    def lonlat_to_xy(longitude, latitude, dx=126, dy=124, BB=(-74.5, -72.8, 40.5, 41.8)):
        return (dx*(longitude - BB[0])/(BB[1]-BB[0])).astype('int'), \
               (dy - dy*(latitude - BB[2])/(BB[3]-BB[2])).astype('int')
    
    pickup_x, pickup_y = lonlat_to_xy(df.pickup_longitude, df.pickup_latitude)
    dropoff_x, dropoff_y = lonlat_to_xy(df.dropoff_longitude, df.dropoff_latitude)
    
    #keep numeric feature only
    df.pop('pickup_datetime')
    df.pop('key')
    df.pop('passenger_count')
    df['time_in_day'] = hour*60.0 + minute
    
    #normalize df
    if mean is None:
        mean = df.mean() 
    if std is None:
        std = df.std()
        
    df = (df-mean)/std
    
    
    
    
    #add new categorical columns
    df['weekday'] = weekday
    df['month'] = month
    df['hour'] = hour
    df['minute'] = minute
    df['dayofyear'] = dayofyear
    
    df['pickup_x'] = pickup_x
    df['pickup_y'] = pickup_y
    df['dropoff_x'] = dropoff_x
    df['dropoff_y'] = dropoff_y
    
    return df, (mean, std)
    

In [440]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.354652,-72.494682,39.914481,-72.490967,39.919053,1.673820
std,9.716777,10.693934,6.225686,10.471386,6.213427,1.300171
min,-44.900000,-736.550000,-74.007670,-84.654241,-74.006377,0.000000
25%,6.000000,-73.992041,40.734996,-73.991215,40.734182,1.000000
50%,8.500000,-73.981789,40.752765,-73.980000,40.753243,1.000000
75%,12.500000,-73.966982,40.767258,-73.963433,40.768166,2.000000
max,200.000000,40.787575,401.083332,40.851027,404.616667,6.000000


In [441]:
df.dtypes

key                               object
fare_amount                      float64
pickup_datetime      datetime64[ns, UTC]
pickup_longitude                 float64
pickup_latitude                  float64
dropoff_longitude                float64
dropoff_latitude                 float64
passenger_count                    int64
dtype: object

In [442]:
df, (mean, std) = preprocess(df)
target = df.pop('fare_amount')
df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,time_in_day,weekday,month,hour,minute,dayofyear,pickup_x,pickup_y,dropoff_x,dropoff_y
0,3.362122,-0.992318,3.494980,-1.187188,0.529124,0,5,17,26,166,48,102,48,103
1,-1.055185,-1.324482,-0.140327,0.915968,0.442096,1,0,16,52,5,35,103,38,97
2,-0.198407,0.332590,-0.456539,-0.032422,-2.058698,3,7,0,35,230,38,99,37,100
3,-0.311375,-0.600195,-0.465122,0.194707,-1.457176,5,3,4,30,112,38,101,37,99
4,0.178231,0.556045,0.456843,0.968995,-0.942683,1,2,7,51,68,39,98,40,96


In [443]:
df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,time_in_day,weekday,month,hour,minute,dayofyear,pickup_x,pickup_y,dropoff_x,dropoff_y
count,9.784000e+04,9.784000e+04,9.784000e+04,9.784000e+04,9.784000e+04,97840.000000,97840.000000,97840.000000,97840.000000,97840.000000,97840.000000,97840.000000,97840.000000,97840.000000
mean,1.138859e-11,1.227544e-11,2.053007e-11,-4.906646e-12,-2.933966e-17,3.036386,5.262807,13.494665,29.603904,175.064146,38.400041,99.538399,38.478475,99.500848
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.949297,3.452205,6.504696,17.300568,105.237391,2.904730,2.901775,2.830043,3.179998
min,-1.191435e+01,-8.330469e+00,-1.322105e+01,-7.588774e+00,-2.148286e+00,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,10.000000,1.000000,8.000000
25%,-4.432483e-01,-4.829957e-01,-4.605007e-01,-4.781426e-01,-7.046344e-01,1.000000,2.000000,9.000000,15.000000,84.000000,37.000000,98.000000,37.000000,98.000000
50%,-1.813663e-01,7.638450e-02,-1.711587e-01,7.060053e-02,9.654088e-02,3.000000,5.000000,14.000000,30.000000,167.000000,38.000000,99.000000,38.000000,99.000000
75%,1.731578e-01,5.453828e-01,2.329866e-01,5.082905e-01,8.465229e-01,5.000000,8.000000,19.000000,45.000000,268.000000,39.000000,101.000000,39.000000,101.000000
max,2.287346e+01,3.121290e+01,2.345555e+01,2.904700e+01,1.535073e+00,6.000000,11.000000,23.000000,59.000000,366.000000,104.000000,123.000000,104.000000,123.000000


In [444]:
input_fn_train = tf.estimator.inputs.pandas_input_fn(x=df, y=target, batch_size=32, num_epochs=100, shuffle=True, target_column="fare_amount")

In [445]:
def build_feature_columns():
    pickup_longitude = tf.feature_column.numeric_column('pickup_longitude')
    pickup_latitude = tf.feature_column.numeric_column('pickup_latitude')
    dropoff_longitude = tf.feature_column.numeric_column('dropoff_longitude')
    dropoff_latitude = tf.feature_column.numeric_column('dropoff_latitude')
    
    base_columns = [

    ]
    
    weekday = tf.feature_column.categorical_column_with_identity('weekday', 7)
    month = tf.feature_column.categorical_column_with_identity('month', 12)
    hour = tf.feature_column.categorical_column_with_identity('hour', 24)
    minute = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('minute'), list(range(0,60,10)))
    
    pickup_x = tf.feature_column.categorical_column_with_identity('pickup_x', 126)
    pickup_y = tf.feature_column.categorical_column_with_identity('pickup_y', 124)
    dropoff_x = tf.feature_column.categorical_column_with_identity('dropoff_x', 126)
    dropoff_y = tf.feature_column.categorical_column_with_identity('dropoff_y', 124)
    
    cross_columns = [
        tf.feature_column.crossed_column([pickup_x, pickup_y], 126*124),
        tf.feature_column.crossed_column([dropoff_x, dropoff_y], 126*124),
        tf.feature_column.crossed_column([month, weekday, hour, minute], 7*12*24*6)
    ]
    
    wide_columns = base_columns + cross_columns
    
    time_in_day = tf.feature_column.numeric_column('time_in_day')
    dayofyear = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('dayofyear'), list(range(1, 365, 7)))
    deep_columns = [
        tf.feature_column.numeric_column('pickup_longitude'),
        tf.feature_column.numeric_column('pickup_latitude'),
        tf.feature_column.numeric_column('dropoff_longitude'),
        tf.feature_column.numeric_column('dropoff_latitude'),
        embedding_column(dayofyear, dimension=16),
        embedding_column(tf.feature_column.bucketized_column(time_in_day, boundaries=list(range(0,24*60, 15))), dimension=32)
    ]
    
    return wide_columns, deep_columns

In [446]:
# feature_columns = [
#     tf.feature_column.numeric_column('pickup_longitude'),
#     tf.feature_column.numeric_column('pickup_latitude'),
#     tf.feature_column.numeric_column('dropoff_longitude'),
#     tf.feature_column.numeric_column('dropoff_latitude'),
#     tf.feature_column.numeric_column('passenger_count'),
#     embedding_column(categorical_column_with_identity('weekday', num_buckets=7), dimension=3),
#     embedding_column(categorical_column_with_identity('month', num_buckets=12), dimension=3),
#     embedding_column(categorical_column_with_identity('hour', num_buckets=24), dimension=5)
# ]
# feature_columns

In [447]:
wide_columns, deep_columns = build_feature_columns()
print(wide_columns, deep_columns)

[CrossedColumn(keys=(IdentityCategoricalColumn(key='pickup_x', number_buckets=126, default_value=None), IdentityCategoricalColumn(key='pickup_y', number_buckets=124, default_value=None)), hash_bucket_size=15624, hash_key=None), CrossedColumn(keys=(IdentityCategoricalColumn(key='dropoff_x', number_buckets=126, default_value=None), IdentityCategoricalColumn(key='dropoff_y', number_buckets=124, default_value=None)), hash_bucket_size=15624, hash_key=None), CrossedColumn(keys=(IdentityCategoricalColumn(key='month', number_buckets=12, default_value=None), IdentityCategoricalColumn(key='weekday', number_buckets=7, default_value=None), IdentityCategoricalColumn(key='hour', number_buckets=24, default_value=None), BucketizedColumn(source_column=NumericColumn(key='minute', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(0, 10, 20, 30, 40, 50))), hash_bucket_size=12096, hash_key=None)] [NumericColumn(key='pickup_longitude', shape=(1,), default_value=None, dtype=t

In [448]:
def build_estimator(fc_fn):
    wide_columns, deep_columns = fc_fn()
    hidden_units = [100, 75, 50, 25]
    return tf.estimator.DNNLinearCombinedRegressor(
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units)
est = build_estimator(build_feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmppcb_mt74', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb31537c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [449]:
# est = tf.estimator.DNNRegressor(
#     feature_columns=feature_columns, 
#     hidden_units=[1024, 512, 256],
#     optimizer=tf.train.AdamOptimizer()
# )



In [450]:
est.train(input_fn=input_fn_train)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmppcb_mt74/model.ckpt.
INFO:tensorflow:loss = 20.849318, step = 1
INFO:tensorflow:global_step/sec: 162.069
INFO:tensorflow:loss = 5.3621035, step = 101 (0.618 sec)
INFO:tensorflow:global_step/sec: 373.024
INFO:tensorflow:loss = 36.84134, step = 201 (0.269 sec)
INFO:tensorflow:global_step/sec: 390.486
INFO:tensorflow:loss = 9.915201, step = 301 (0.259 sec)
INFO:tensorflow:global_step/sec: 240.436
INFO:tensorflow:loss = 6.48221, step = 401 (0.416 sec)
INFO:tensorflow:global_step/sec: 307.806
INFO:tensorflow:loss = 5.5415087, step = 501 (0.323 sec)
INFO:tensorflow:global_step/sec: 317.748
INFO:tensorflow:loss = 3.2901518, step = 601 (0.316 sec)
INFO:tensorflow:g

INFO:tensorflow:loss = 1.6310046, step = 8001 (0.278 sec)
INFO:tensorflow:global_step/sec: 327.267
INFO:tensorflow:loss = 3.0436306, step = 8101 (0.306 sec)
INFO:tensorflow:global_step/sec: 286.557
INFO:tensorflow:loss = 5.195569, step = 8201 (0.351 sec)
INFO:tensorflow:global_step/sec: 129.487
INFO:tensorflow:loss = 8.318304, step = 8301 (0.773 sec)
INFO:tensorflow:global_step/sec: 219.402
INFO:tensorflow:loss = 5.9387646, step = 8401 (0.455 sec)
INFO:tensorflow:global_step/sec: 362.9
INFO:tensorflow:loss = 2.3582177, step = 8501 (0.274 sec)
INFO:tensorflow:global_step/sec: 433.55
INFO:tensorflow:loss = 2.5793495, step = 8601 (0.234 sec)
INFO:tensorflow:global_step/sec: 445.677
INFO:tensorflow:loss = 7.007881, step = 8701 (0.222 sec)
INFO:tensorflow:global_step/sec: 436.929
INFO:tensorflow:loss = 2.751358, step = 8801 (0.229 sec)
INFO:tensorflow:global_step/sec: 364.048
INFO:tensorflow:loss = 2.6020985, step = 8901 (0.276 sec)
INFO:tensorflow:global_step/sec: 368.355
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 436.64
INFO:tensorflow:loss = 30.619347, step = 16301 (0.227 sec)
INFO:tensorflow:global_step/sec: 364.904
INFO:tensorflow:loss = 5.05114, step = 16401 (0.274 sec)
INFO:tensorflow:global_step/sec: 379.299
INFO:tensorflow:loss = 13.104796, step = 16501 (0.264 sec)
INFO:tensorflow:global_step/sec: 324.853
INFO:tensorflow:loss = 70.18089, step = 16601 (0.308 sec)
INFO:tensorflow:global_step/sec: 375.042
INFO:tensorflow:loss = 8.136548, step = 16701 (0.268 sec)
INFO:tensorflow:global_step/sec: 389.701
INFO:tensorflow:loss = 2.0030658, step = 16801 (0.257 sec)
INFO:tensorflow:global_step/sec: 416.148
INFO:tensorflow:loss = 2.089639, step = 16901 (0.240 sec)
INFO:tensorflow:global_step/sec: 416.835
INFO:tensorflow:loss = 2.031234, step = 17001 (0.240 sec)
INFO:tensorflow:global_step/sec: 339.825
INFO:tensorflow:loss = 3.6918476, step = 17101 (0.293 sec)
INFO:tensorflow:global_step/sec: 325.819
INFO:tensorflow:loss = 1.566135, step = 17201 (0.309 sec)
INFO:ten

INFO:tensorflow:loss = 4.658469, step = 24501 (0.302 sec)
INFO:tensorflow:global_step/sec: 333.395
INFO:tensorflow:loss = 22.38292, step = 24601 (0.302 sec)
INFO:tensorflow:global_step/sec: 285.262
INFO:tensorflow:loss = 5.3191223, step = 24701 (0.353 sec)
INFO:tensorflow:global_step/sec: 126.697
INFO:tensorflow:loss = 1.7156396, step = 24801 (0.786 sec)
INFO:tensorflow:global_step/sec: 233.64
INFO:tensorflow:loss = 2.511625, step = 24901 (0.432 sec)
INFO:tensorflow:global_step/sec: 334.682
INFO:tensorflow:loss = 31.496056, step = 25001 (0.296 sec)
INFO:tensorflow:global_step/sec: 331.782
INFO:tensorflow:loss = 2.6005416, step = 25101 (0.304 sec)
INFO:tensorflow:global_step/sec: 303.248
INFO:tensorflow:loss = 3.5553432, step = 25201 (0.327 sec)
INFO:tensorflow:global_step/sec: 278.95
INFO:tensorflow:loss = 5.9225583, step = 25301 (0.361 sec)
INFO:tensorflow:global_step/sec: 271.437
INFO:tensorflow:loss = 4.031417, step = 25401 (0.371 sec)
INFO:tensorflow:global_step/sec: 379.054
INFO:t

INFO:tensorflow:global_step/sec: 268.882
INFO:tensorflow:loss = 19.219366, step = 32801 (0.369 sec)
INFO:tensorflow:global_step/sec: 189.757
INFO:tensorflow:loss = 51.91261, step = 32901 (0.527 sec)
INFO:tensorflow:global_step/sec: 426.034
INFO:tensorflow:loss = 3.6889954, step = 33001 (0.235 sec)
INFO:tensorflow:global_step/sec: 315.355
INFO:tensorflow:loss = 4.4923525, step = 33101 (0.315 sec)
INFO:tensorflow:global_step/sec: 339.111
INFO:tensorflow:loss = 3.6726298, step = 33201 (0.299 sec)
INFO:tensorflow:global_step/sec: 335.21
INFO:tensorflow:loss = 4.012218, step = 33301 (0.296 sec)
INFO:tensorflow:global_step/sec: 249.436
INFO:tensorflow:loss = 12.264594, step = 33401 (0.401 sec)
INFO:tensorflow:global_step/sec: 370.678
INFO:tensorflow:loss = 1.1378894, step = 33501 (0.268 sec)
INFO:tensorflow:global_step/sec: 423.003
INFO:tensorflow:loss = 2.1158352, step = 33601 (0.241 sec)
INFO:tensorflow:global_step/sec: 416.899
INFO:tensorflow:loss = 4.465575, step = 33701 (0.237 sec)
INFO

...

INFO:tensorflow:global_step/sec: 367.56
INFO:tensorflow:loss = 1.6391168, step = 245901 (0.272 sec)
INFO:tensorflow:global_step/sec: 451.463
INFO:tensorflow:loss = 2.114903, step = 246001 (0.222 sec)
INFO:tensorflow:global_step/sec: 411.686
INFO:tensorflow:loss = 3.3055973, step = 246101 (0.243 sec)
INFO:tensorflow:global_step/sec: 384.614
INFO:tensorflow:loss = 1.2054157, step = 246201 (0.260 sec)
INFO:tensorflow:global_step/sec: 333.611
INFO:tensorflow:loss = 93.594696, step = 246301 (0.303 sec)
INFO:tensorflow:global_step/sec: 363.576
INFO:tensorflow:loss = 3.779872, step = 246401 (0.650 sec)
INFO:tensorflow:global_step/sec: 130.89
INFO:tensorflow:loss = 3.770485, step = 246501 (0.389 sec)
INFO:tensorflow:global_step/sec: 350.141
INFO:tensorflow:loss = 1.989044, step = 246601 (0.285 sec)
INFO:tensorflow:global_step/sec: 388.984
INFO:tensorflow:loss = 1.0566255, step = 246701 (0.258 sec)
INFO:tensorflow:global_step/sec: 401.834
INFO:tensorflow:loss = 1.2967923, step = 246801 (0.248 s

INFO:tensorflow:global_step/sec: 377.044
INFO:tensorflow:loss = 2.4183135, step = 254101 (0.264 sec)
INFO:tensorflow:global_step/sec: 388.701
INFO:tensorflow:loss = 3.8039644, step = 254201 (0.260 sec)
INFO:tensorflow:global_step/sec: 402.943
INFO:tensorflow:loss = 2.3149495, step = 254301 (0.246 sec)
INFO:tensorflow:global_step/sec: 362.444
INFO:tensorflow:loss = 24.148542, step = 254401 (0.275 sec)
INFO:tensorflow:global_step/sec: 355.816
INFO:tensorflow:loss = 2.175113, step = 254501 (0.281 sec)
INFO:tensorflow:global_step/sec: 389.147
INFO:tensorflow:loss = 7.476462, step = 254601 (0.259 sec)
INFO:tensorflow:global_step/sec: 332.897
INFO:tensorflow:loss = 1.5304681, step = 254701 (0.300 sec)
INFO:tensorflow:global_step/sec: 453.325
INFO:tensorflow:loss = 3.0593634, step = 254801 (0.220 sec)
INFO:tensorflow:global_step/sec: 333.814
INFO:tensorflow:loss = 7.43938, step = 254901 (0.301 sec)
INFO:tensorflow:global_step/sec: 271.788
INFO:tensorflow:loss = 2.2855554, step = 255001 (0.367

INFO:tensorflow:loss = 1.3318744, step = 262201 (0.258 sec)
INFO:tensorflow:global_step/sec: 398.667
INFO:tensorflow:loss = 12.409931, step = 262301 (0.251 sec)
INFO:tensorflow:global_step/sec: 384.474
INFO:tensorflow:loss = 1.1430681, step = 262401 (0.260 sec)
INFO:tensorflow:global_step/sec: 454.44
INFO:tensorflow:loss = 1.6489348, step = 262501 (0.221 sec)
INFO:tensorflow:global_step/sec: 341.183
INFO:tensorflow:loss = 2.2284565, step = 262601 (0.292 sec)
INFO:tensorflow:global_step/sec: 334.885
INFO:tensorflow:loss = 19.977354, step = 262701 (0.299 sec)
INFO:tensorflow:global_step/sec: 142.207
INFO:tensorflow:loss = 2.800661, step = 262801 (0.705 sec)
INFO:tensorflow:global_step/sec: 273.46
INFO:tensorflow:loss = 1.8836551, step = 262901 (0.366 sec)
INFO:tensorflow:global_step/sec: 349.729
INFO:tensorflow:loss = 2.7295475, step = 263001 (0.283 sec)
INFO:tensorflow:global_step/sec: 458.671
INFO:tensorflow:loss = 2.3660755, step = 263101 (0.221 sec)
INFO:tensorflow:global_step/sec: 3

INFO:tensorflow:global_step/sec: 342.843
INFO:tensorflow:loss = 2.1202974, step = 270401 (0.292 sec)
INFO:tensorflow:global_step/sec: 326.775
INFO:tensorflow:loss = 15.48115, step = 270501 (0.305 sec)
INFO:tensorflow:global_step/sec: 402.856
INFO:tensorflow:loss = 3.1199632, step = 270601 (0.248 sec)
INFO:tensorflow:global_step/sec: 346.353
INFO:tensorflow:loss = 6.580782, step = 270701 (0.291 sec)
INFO:tensorflow:global_step/sec: 352.259
INFO:tensorflow:loss = 10.186684, step = 270801 (0.284 sec)
INFO:tensorflow:global_step/sec: 402.442
INFO:tensorflow:loss = 3.3141794, step = 270901 (0.248 sec)
INFO:tensorflow:global_step/sec: 351.889
INFO:tensorflow:loss = 6.126323, step = 271001 (0.281 sec)
INFO:tensorflow:global_step/sec: 260.692
INFO:tensorflow:loss = 2.3936496, step = 271101 (0.387 sec)
INFO:tensorflow:global_step/sec: 247.335
INFO:tensorflow:loss = 1.9787143, step = 271201 (0.402 sec)
INFO:tensorflow:global_step/sec: 195.549
INFO:tensorflow:loss = 1.529685, step = 271301 (0.511

KeyboardInterrupt: 

In [453]:
TEST_PATH = "/Users/bryanwu/Desktop/taxi_fare_prediction/new-york-city-taxi-fare-prediction/train.csv"
df_test = pd.read_csv(TEST_PATH, skiprows=range(1,1_000_000), parse_dates=['pickup_datetime'], nrows=2000)
df_test.head()
df_test, _ = preprocess(df_test, mean, std)
target_test = df_test.pop('fare_amount')

In [454]:
input_fn_test = tf.estimator.inputs.pandas_input_fn(x=df_test, 
                                                     y=target_test,  
                                                    shuffle=False,
                                                    batch_size=2000,
                                                     target_column="fare_amount"
                                                    )

In [455]:
predictions = est.predict(
    input_fn_test,
    predict_keys=None,
    hooks=None,
    checkpoint_path=None,
    yield_single_examples=False
)

In [456]:
predictions = list(predictions)[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmppcb_mt74/model.ckpt-196141
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [457]:
predictions

{'predictions': array([[-0.62827605],
        [ 0.18001917],
        [-0.6308625 ],
        ...,
        [-0.38880777],
        [ 0.26244384],
        [ 0.7254192 ]], dtype=float32)}

In [458]:
predictions = predictions['predictions']

In [459]:
predictions

array([[-0.62827605],
       [ 0.18001917],
       [-0.6308625 ],
       ...,
       [-0.38880777],
       [ 0.26244384],
       [ 0.7254192 ]], dtype=float32)

In [460]:
mean.astype('float32')
std.astype('float32')

fare_amount            9.682800
pickup_longitude       0.038878
pickup_latitude        0.030154
dropoff_longitude      0.037867
dropoff_latitude       0.033153
time_in_day          390.676025
dtype: float32

In [461]:
predictions = predictions*std['fare_amount']+mean['fare_amount']

In [462]:
predictions

array([[ 5.2599716],
       [13.086533 ],
       [ 5.2349277],
       ...,
       [ 7.578695 ],
       [13.884634 ],
       [18.367533 ]], dtype=float32)

In [463]:
len(predictions)

1967

In [464]:
target_test_tensor = tf.dtypes.cast(tf.constant(target_test), tf.float32)*std['fare_amount']+mean['fare_amount']
target_test_tensor

<tf.Tensor 'add_2:0' shape=(1967,) dtype=float32>

In [465]:
import numpy as np
target_test_tensor.shape
np.squeeze(predictions).shape

(1967,)

In [466]:
rsme = tf.metrics.mean_squared_error(labels=target_test_tensor, predictions=predictions)

In [467]:
from tensorflow import keras
rsme = keras.losses.mean_squared_error(np.squeeze(predictions), target_test_tensor)

In [468]:
session = tf.Session()

In [469]:
np.sqrt(session.run(rsme))

3.9402096

In [470]:
sess.run(target_test_tensor[:10])

array([ 4.1      , 10.099999 ,  3.6999998, 11.       ,  6.9      ,
       20.5      , 16.5      ,  5.7000003, 21.5      ,  6.9      ],
      dtype=float32)

In [471]:
predictions[:10]

array([[ 5.2599716],
       [13.086533 ],
       [ 5.2349277],
       [ 9.329629 ],
       [ 6.6583457],
       [13.105282 ],
       [12.666529 ],
       [ 6.1988425],
       [16.19311  ],
       [ 6.5673385]], dtype=float32)